In [1]:
#encoding: utf-8
from __future__ import division
from nodebox.graphics import *
import pymunk
import pymunk.pyglet_util
import random, math
import numpy as np

space = pymunk.Space()

def createBody(x,y,shape,*shapeArgs):
    body = pymunk.Body()
    body.position = x, y
    s = shape(body, *shapeArgs)
    s.mass = 1
    s.friction = 1
    space.add(body, s)
    return s #shape!!!

s0=createBody(300, 300, pymunk.Poly, ((-20,-5),(-20,5),(20,15),(20,-15)))
s0.score=0
s3=createBody(200, 300, pymunk.Poly, ((-20,-5),(-20,5),(20,15),(20,-15)))
s3.color = (0, 255, 0, 255)
s3.score=0
s3.body.Q=[[0, 0], [0, 0], [0, 0], [0, 0]]  # Додано новий стан
s3.body.action=0 # 0 - залишати, 1 - змінювати (випадковий кут)
s3.body.position_history = []  # Історія позицій
s3.body.lingering = False  # Прапорець нерухомості
s1=createBody(300, 200, pymunk.Circle, 10, (0,0))
S2=[]
for i in range(1):
    s2=createBody(350, 250, pymunk.Circle, 10, (0,0))
    s2.color = (255, 0, 0, 255)
    S2.append(s2)


def getAngle(x,y,x1,y1):
    return math.atan2(y1-y, x1-x)

def getDist(x,y,x1,y1):
    return ((x-x1)**2+(y-y1)**2)**0.5

def inCircle(x,y,cx,cy,R):
    if (x-cx)**2+(y-cy)**2 < R**2:
        return True
    return False

def inSector(x,y,cx,cy,R,a):
    angle=getAngle(cx,cy,x,y)
    a=a%(2*math.pi)
    angle=angle%(2*math.pi)
    if inCircle(x,y,cx,cy,R) and a-0.5<angle<a+0.5:
        return True
    return False

def strategy2(b=s3.body):
    u"""Стратегія робота з покаранням за нерухомість."""
    v=100
    a=b.angle
    b.velocity=v*math.cos(a), v*math.sin(a)
    x,y=b.position
    R=getDist(x,y,350,250)
    ellipse(x, y, 200, 200, stroke=Color(0.5))
    line(x,y,x+100*math.cos(a+0.5),y+100*math.sin(a+0.5),stroke=Color(0.5))
    line(x,y,x+100*math.cos(a-0.5),y+100*math.sin(a-0.5),stroke=Color(0.5))

    # Оновлюємо історію позицій
    if canvas.frame % 10 == 0:
        b.position_history.append((x, y))
        if len(b.position_history) > 500:  # Зберігаємо позиції за останні 10 секунд (500 кадрів * 0.02 с)
            b.position_history.pop(0)

    # Перевірка на нерухомість
    if len(b.position_history) >= 2:
        recent_positions = b.position_history[-250:]  # Останні 5 секунд
        avg_x = sum(pos[0] for pos in recent_positions) / len(recent_positions)
        avg_y = sum(pos[1] for pos in recent_positions) / len(recent_positions)
        avg_dist = getDist(x, y, avg_x, avg_y)
        if avg_dist < 20:  # Якщо агент знаходиться в межах 20 пікселів від середньої позиції
            b.lingering = True
        else:
            b.lingering = False

    if canvas.frame%10==0: # кожні n кадрів
        inS=inSector(s1.body.position[0], s1.body.position[1], x, y, 100, a)
        inS2=inSector(S2[0].body.position[0], S2[0].body.position[1], x, y, 100, a)

        # установлюємо стан і винагороду
        if b.lingering:
            state=3  # Новий стан - нерухомість
            reward=-1  # Штраф за нерухомість
        elif inS:
            state=1
            reward=1 if b.action==0 else -1
        elif inS2:
            state=2
            reward=-1 if b.action==0 else 1
        else:
            state=0; reward=0
        b.Q[state][b.action] += reward  # оновлюємо Q таблицю
        print(f"Стан: {state}, Дія: {b.action}, Q-таблиця: {b.Q}")

        # вибираємо дію
        act = b.Q[state][b.action]
        if random.random() < abs(1.0 / (act + 0.1)):  # 0.1 запобігає /0
            b.action = random.choice([0, 1])  # випадково 50/50
        else:
            b.action = np.argmax(b.Q[state])  # залишати чи змінювати?

        if b.action:  # якщо змінювати
            b.angle = 2 * math.pi * random.random()

        if R > 180:  # запобігти виїзду за межі
            b.angle = getAngle(x, y, 350, 250)

def scr(s,s0,s3,p=1):
    bx,by=s.body.position
    s0x,s0y=s0.body.position
    s3x,s3y=s3.body.position
    if not inCircle(bx,by,350,250,180):
        if getDist(bx,by,s0x,s0y)<getDist(bx,by,s3x,s3y):
            s0.score=s0.score+p
        else:
            s3.score=s3.score+p
        s.body.position=random.randint(200,400),random.randint(200,300)

def score():
    u"""визначає переможця"""
    scr(s1,s0,s3)
    for s in S2:
        scr(s,s0,s3,p=-1)

def manualControl():
    u"""Керування роботом з мишки або клавіатури"""
    v=10  # швидкість
    b=s0.body
    a=b.angle
    x,y=b.position
    vx,vy=b.velocity
    if canvas.keys.char=="a":
        b.angle-=0.1
    if canvas.keys.char=="d":
        b.angle+=0.1
    if canvas.keys.char=="w":
        b.velocity=vx+v*math.cos(a), vy+v*math.sin(a)
    if canvas.mouse.button==LEFT:
        b.angle=getAngle(x,y,*canvas.mouse.xy)
        b.velocity=vx+v*math.cos(a), vy+v*math.sin(a)

def simFriction():
    for s in [s0,s1,s3]+S2:
        s.body.velocity=s.body.velocity[0]*0.9, s.body.velocity[1]*0.9
        s.body.angular_velocity=s.body.angular_velocity*0.9

draw_options = pymunk.pyglet_util.DrawOptions()

def draw(canvas):
    canvas.clear()
    fill(0,0,0,1)
    text("%i %i"%(s0.score,s3.score),20,20)
    nofill()
    ellipse(350, 250, 350, 350, stroke=Color(0))
    manualControl()
    strategy2()
    score()
    simFriction()
    space.step(0.02)
    space.debug_draw(draw_options)

canvas.size = 700, 500
canvas.run(draw)

ModuleNotFoundError: No module named 'context'